# References
https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/

# Install and Import Libraries

In [ ]:
!pip install pmdarima

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, acf
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from pmdarima.arima.utils import ndiffs

# Box-Jenkins Approach
1. Identification: Determine the order of the parameters of SARIMA(p,d,q)(P,D,Q)s
2. Estimation: Train the parameters of the model.
3. Diagnostic checking: Evaluate the fitted model.

# Finding the best ARIMA model

## Loading and Ploting the Data

In [ ]:
# Import WWWUsage data (The numbers of users connected to the internet through a server every minute)

df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/wwwusage.csv', names=['value'], header=0)

In [ ]:
# Create Training and Test
train = df['value'][:85]
test = df['value'][85:]

plt.figure(figsize=(7, 3))
plt.plot(train, label='train')
plt.plot(test, label='test')
plt.ylabel('The Number of Users')
plt.xlabel('Time (min)')
plt.legend()
plt.show()

In [ ]:
# Build a ARIMA Model
model = ARIMA(train, order=(1, 1, 1))
fitted = model.fit()  

# Forecast
forecast = fitted.get_forecast(len(test), alpha=0.05)  # 95% conf
fc = forecast.predicted_mean
conf = forecast.conf_int()

# Compare the forecast data and the actual data
def plot_forecast(train: pd.Series, test: pd.Series, fc: pd.Series, conf: pd.DataFrame):
    # Make as pandas series
    fc_series = pd.Series(fc, index=test.index)
    lower_series = pd.Series(conf.iloc[:, 0], index=test.index)
    upper_series = pd.Series(conf.iloc[:, 1], index=test.index)

    # Plot
    plt.figure(figsize=(7, 3))
    plt.plot(train, label='training')
    plt.plot(test, label='actual')
    plt.plot(fc_series, label='forecast')
    plt.fill_between(lower_series.index, lower_series, upper_series, 
                    color='k', alpha=.15)
    plt.xlabel('Time (min)')
    plt.ylabel('The Number of Users')
    plt.title('Forecast vs Actuals')
    plt.legend(loc='upper left', fontsize=8)
    plt.tight_layout()
    plt.show()

plot_forecast(train, test, fc, conf)

In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast: pd.Series, actual: pd.Series):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast.to_numpy()[:,None], 
                              actual.to_numpy()[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast.to_numpy()[:,None], 
                              actual.to_numpy()[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(fc-test)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(fc, test)

## Residual Diagnostics

In [ ]:
def plot_residual(train: pd.Series, prediction: pd.Series):
  residual = train - prediction
  fig, ax = plt.subplots(2, 2, figsize=(14, 5))
  ax[0, 0].plot(prediction, label='fitted')
  ax[0, 0].plot(train, label='train')
  ax[0, 0].legend()
  ax[0, 0].set_title('Train data vs Fitted values')
  ax[0, 0].set_xlabel('Time (min)')
  ax[0, 0].set_ylabel('The Number of Users')
  ax[0, 1].plot(residual)
  ax[0, 1].set_title('Residuals')
  ax[0, 1].set_xlabel('Time (min)')
  ax[0, 1].set_ylabel('The Number of Users')
  plot_acf(residual, ax=ax[1, 0])
  ax[1, 0].set_xlabel('Lags')
  plot_pacf(residual, ax=ax[1, 1], method='ywm')
  ax[1, 1].set_xlabel('Lags')
  plt.tight_layout()
  plt.show()

In [ ]:
p, d, q = 1, 1, 1
model = ARIMA(train, order=(p, d, q))
model_fit = model.fit()
prediction = model_fit.get_prediction().predicted_mean[d:]
forecast = model_fit.get_forecast(len(test), alpha=0.05)  # 95% conf
fc = forecast.predicted_mean
conf = forecast.conf_int()
plot_residual(train[d:], prediction)
plot_forecast(train, test, fc, conf)
print(model_fit.summary())

## Detemining the order of differencing (d) (Manual)



In [ ]:
y = df['value']
fig, axes = plt.subplots(4, 3, figsize=(17, 7))
for i in range(4):
    result = adfuller(y)
    axes[i, 0].plot(y); axes[i, 0].set_title(
        f'Differencing order: {i}, ADF Statistic: {result[0]:.02f}, p-value: {result[1]:.02f}')
    axes[i, 0].set_xlabel('Time (min)')
    axes[i, 0].set_ylabel('# of Users')
    plot_acf(y, ax=axes[i, 1])
    axes[i, 1].set_xlabel('Lags')
    plot_pacf(y, ax=axes[i, 2], method='ywm')
    axes[i, 2].set_xlabel('Lags')
    y = y.diff().dropna()
plt.tight_layout()
plt.show()

## Exercise: Find the optimal order of differencing
Hint: Use ```plot_residual```, ```plot_forecast```, ```model_fit.summary()```

In [ ]:
p, d, q = 0, 1, 0
model = ARIMA(train, order=(p, d, q))
model_fit = model.fit()
prediction = model_fit.get_prediction().predicted_mean[d:]
forecast = model_fit.get_forecast(len(test), alpha=0.05)  # 95% conf
fc = forecast.predicted_mean
conf = forecast.conf_int()
plot_residual(train[d:], prediction)
plot_forecast(train, test, fc, conf)
print(model_fit.summary())

In [ ]:
# TODO

## Detemining the order of differencing (d) (Automatic)

In [ ]:
# ADF test
print(ndiffs(df.value, test='adf'))

# KPSS test
print(ndiffs(df.value, test='kpss'))

## Exercise: Find the opotimal order of AR(p) and MA(q)

## Determining AR(p) and MA(q) parameters (Manual)


In [ ]:
# We starts from ARIMA(0, 2, 0)

p, d, q = 0, 2, 0
model = ARIMA(train, order=(p, d, q))
model_fit = model.fit()
prediction = model_fit.get_prediction().predicted_mean[d:]
forecast = model_fit.get_forecast(len(test), alpha=0.05)  # 95% conf
fc = forecast.predicted_mean
conf = forecast.conf_int()
plot_residual(train[d:], prediction)
plot_forecast(train, test, fc, conf)
print(model_fit.summary())

In [ ]:
# TODO: Try to remove lag-2 ACF and lag-2 PACF

## Determining AR(p) and MA(q) parameters (Automatic)

In [ ]:
model = pm.auto_arima(train, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

p, d, q = model.get_params()['order']
prediction = pd.Series(model.predict_in_sample(), index=train.index).iloc[d:]
fc, conf = model.predict(15, return_conf_int=True)
forecast_timestamp = np.arange(train.index[-1] + 1, train.index[-1] + 16)
fc = pd.Series(fc, index=forecast_timestamp)
conf = pd.DataFrame(conf, index=forecast_timestamp)
plot_residual(train[d:], prediction)
plot_forecast(train, test, fc, conf)
print(model.summary())

In [ ]:
forecast_accuracy(fc, test)

# Finding the best SARIMA model

In [ ]:
# Import
data = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/a10.csv', parse_dates=['date'], index_col='date')
train = data['value'][:180]
test = data['value'][180:]
train.index = pd.DatetimeIndex(train.index.values, freq='MS')
test.index = pd.DatetimeIndex(test.index.values, freq='MS')

## Seasonal Differencing

In [ ]:
# Plot
fig, axes = plt.subplots(2, 1, figsize=(10,5), dpi=100, sharex=True)

# Usual Differencing
axes[0].plot(data[:], label='Original Series')
axes[0].plot(data[:].diff(1), label='Usual Differencing')
axes[0].set_title('Usual Differencing')
axes[0].legend(loc='upper left', fontsize=10)

# Seasinal Differencing
axes[1].plot(data[:], label='Original Series')
axes[1].plot(data[:].diff(12), label='Seasonal Differencing', color='green')
axes[1].set_title('Seasonal Differencing')
plt.legend(loc='upper left', fontsize=10)
plt.suptitle('a10 - Drug Sales', fontsize=16)
plt.show()

In [ ]:
p, d, q = 1, 1, 1
P, D, Q, s = 0, 0, 0, 12
model = ARIMA(train, order=(p, d, q), seasonal_order=(P, D, Q, s))
model_fit = model.fit()
prediction = model_fit.get_prediction().predicted_mean[d + D * s:]
forecast = model_fit.get_forecast(len(test), alpha=0.05)  # 95% conf
fc = forecast.predicted_mean
conf = forecast.conf_int()
plot_residual(train[d + D * s:], prediction)
plot_forecast(train, test, fc, conf)
print(model_fit.summary())

## Exercise: Find the best SARIMA model for the given data

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

In [ ]:
# TODO: Run residual diagnosis on the model